<a href="https://colab.research.google.com/github/cornguo/colab/blob/main/BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install transformers

     |████████████████████████████████| 1.3MB 4.9MB/s 
     |████████████████████████████████| 2.9MB 30.9MB/s 
     |████████████████████████████████| 1.1MB 38.1MB/s 
     |████████████████████████████████| 890kB 54.5MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=521c345b56c9e186a2130c475dc5fb298f5c1cfa8693ccc6c51a94016ac953af
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [5]:
import torch
from transformers import BertTokenizer
from IPython.display import clear_output

In [6]:
PRETRAINED_MODEL_NAME = "bert-base-chinese"  # 指定繁簡中文 BERT-BASE 預訓練模型

# 取得此預訓練模型所使用的 tokenizer
tokenizer = BertTokenizer.from_pretrained(PRETRAINED_MODEL_NAME)

In [7]:
!wget https://raw.githubusercontent.com/cornguo/colab/main/news.zip
!unzip news.zip

--2020-11-08 18:47:35--  https://raw.githubusercontent.com/cornguo/colab/main/news.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 404812 (395K) [application/zip]
Saving to: ‘news.zip.1’

news.zip.1          100%[===================>] 395.32K  --.-KB/s    in 0.04s   

2020-11-08 18:47:35 (10.9 MB/s) - ‘news.zip.1’ saved [404812/404812]

Archive:  news.zip
replace news/test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: a
error:  invalid response [a]
replace news/test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: news/test.csv           
  inflating: news/train.csv          
  inflating: news/valid.csv          


In [8]:
"""
前處理原始的訓練數據集。
你不需了解細節，只需要看註解了解邏輯或是輸出的數據格式即可
"""
import os
import pandas as pd


# 簡單的數據清理，去除空白標題的 examples
df_train = pd.read_csv("/content/news/train.csv")
df_train = df_train[['text', 'keyword']]

# 剔除過長的樣本以避免 BERT 無法將整個輸入序列放入記憶體不多的 GPU
MAX_LENGTH = 250
df_train['text'] = df_train['text'].apply(lambda x: x[:MAX_LENGTH])

# 去除不必要的欄位並重新命名兩標題的欄位名
df_train = df_train.reset_index()
df_train = df_train.loc[:, ['text', 'keyword']]

# idempotence, 將處理結果另存成 tsv 供 PyTorch 使用
df_train.to_csv("train_news.tsv", sep="\t", index=False)

print("訓練樣本數：", len(df_train))
df_train.head()

訓練樣本數： 282


,text,keyword
0,又有從台北飛曼谷班機旅客發燒，疫情指揮中心發言人莊人祥今天說，目前掌握9名泰國旅客到泰國後有...,疫情
1,根據調查，退休準備的年齡從2016年的43.29歲降至今年(2020年)的37.87歲，下修...,投資
2,今（21）日為三倍券上路第7天，根據經濟部統計，截至16點止，目前已經累積1225.8萬人領...,三倍券
3,針對有媒體報導，台北市長柯文哲動支第二預備金買下國民黨南港民服社1.21坪，遭質疑獨厚國民黨...,政治
4,午後對流發展旺盛，易有短時強降雨，今（21）日南部、南投地區及嘉義、雲林山區有局部大雨發生的...,天氣


In [9]:
df_train.keyword.value_counts() / len(df_train)

社會新聞    0.443262
政治      0.226950
疫情      0.102837
名人      0.067376
三倍券     0.046099
生活      0.042553
車       0.028369
投資      0.021277
天氣      0.017730
大自然     0.003546
Name: keyword, dtype: float64

In [10]:
df_test = pd.read_csv("/content/news/valid.csv")
df_test = df_test.loc[:, ["text",'id']]

# 剔除過長的樣本以避免 BERT 無法將整個輸入序列放入記憶體不多的 GPU
MAX_LENGTH = 250
df_test['text'] = df_test['text'].apply(lambda x: x[:MAX_LENGTH])

df_test.to_csv("test_news.tsv", sep="\t", index=False)

print("預測樣本數：", len(df_test))
df_test.head()

預測樣本數： 18


,text,id
0,台灣今年夏天相當炎熱，屢創歷史紀錄，冷氣開幾度最舒適成為民眾關心焦點。有網友透露，自己剛開冷...,283
1,立法院臨時會三讀通過「國民法官法」，民進黨立委在多數優勢下，花了三天兩夜進行113條的審查和...,284
2,新北市新莊重新堤外道，21日晚間發生一起死亡車禍，一名56歲保全騎車要去跟朋友聚餐，卻遭到一...,285
3,凌晨12點多，台北市內湖一名許姓男子在住家門口和朋友聊天，突然看到三台轎車朝他開過來，發覺不...,286
4,日本厚生勞動省已批准將廉價、廣為使用的類固醇藥物「地塞米松」，作為2019冠狀病毒疾病（CO...,287


In [11]:
len(df_test.iloc[3,].text)

250

In [12]:
"""
實作一個可以用來讀取訓練 / 測試集的 Dataset，這是你需要徹底了解的部分。
此 Dataset 每次將 tsv 裡的一筆成對句子轉換成 BERT 相容的格式，並回傳 3 個 tensors：
- tokens_tensor：兩個句子合併後的索引序列，包含 [CLS] 與 [SEP]
- segments_tensor：可以用來識別兩個句子界限的 binary tensor
- label_tensor：將分類標籤轉換成類別索引的 tensor, 如果是測試集則回傳 None
"""
from torch.utils.data import Dataset
 
    
class NewsDataset(Dataset):
    # 讀取前處理後的 tsv 檔並初始化一些參數
    def __init__(self, mode, tokenizer):
        assert mode in ["train_news", "test_news"]  # 一般訓練你會需要 dev set
        self.mode = mode
        # 大數據你會需要用 iterator=True
        self.df = pd.read_csv(mode + ".tsv", sep="\t").fillna("")
        self.len = len(self.df)
        self.label_map = {'疫情': 0, '投資': 1, '三倍券': 2, '政治': 3, '天氣': 4, '大自然': 5, '社會新聞': 6, '生活': 7, '名人': 8, '車': 9}
        self.tokenizer = tokenizer  # 我們將使用 BERT tokenizer
    
    # 定義回傳一筆訓練 / 測試數據的函式
    def __getitem__(self, idx):
        if self.mode == "test_news":
            #text= self.df.iloc[idx, 0].values
            text= self.df.iloc[idx, 0]
            label_tensor = None
        else:
            text, label = self.df.iloc[idx, :].values
            # 將 label 文字也轉換成索引方便轉換成 tensor
            label_id = self.label_map[label]
            label_tensor = torch.tensor(label_id)
            
        # 建立第一個句子的 BERT tokens 並加入分隔符號 [SEP]
        word_pieces = ["[CLS]"]
        tokens = self.tokenizer.tokenize(text)
        word_pieces += tokens + ["[SEP]"]
        len_a = len(word_pieces)
        
        # 將整個 token 序列轉換成索引序列
        ids = self.tokenizer.convert_tokens_to_ids(word_pieces)
        tokens_tensor = torch.tensor(ids)
        
        # 將第一句包含 [SEP] 的 token 位置設為 0，其他為 1 表示第二句
        segments_tensor = torch.tensor([0] * len_a, 
                                        dtype=torch.long)
        
        return (tokens_tensor, segments_tensor, label_tensor)
    
    def __len__(self):
        return self.len
    
    
# 初始化一個專門讀取訓練樣本的 Dataset，使用中文 BERT 斷詞
trainset = NewsDataset("train_news", tokenizer=tokenizer)

In [13]:
# 選擇第一個樣本
sample_idx = 5

# 將原始文本拿出做比較
text, label = trainset.df.iloc[sample_idx].values

# 利用剛剛建立的 Dataset 取出轉換後的 id tensors
tokens_tensor, segments_tensor, label_tensor = trainset[sample_idx]

# 將 tokens_tensor 還原成文本
tokens = tokenizer.convert_ids_to_tokens(tokens_tensor.tolist())
combined_text = "".join(tokens)

# 渲染前後差異，毫無反應就是個 print。可以直接看輸出結果
print(f"""[原始文本]
句子：{text}
分類：{label}
""")

[原始文本]
句子：新海研1號今天交船，台灣大氣與海洋研究再添生力軍。台大海研所所長詹森表示，規劃年底首航關島，1年內裝設大氣觀測雷達，讓新海1升級為國際級海研船。「新海研1號」由科技部委託台灣國際造船公司設計建造，歷時1年多，今天在台船基隆廠舉辦交船典禮，取代已服役34年的「海研一號」，將專注在大氣與海洋研究。新海1總噸位2155噸，可乘載47人，具備超過40天油水及食物的遠洋研究能力。新海研1號將由現有海洋研究船的管理營運學校台灣大學繼續管理營運。負責新海1營運的台大海研所所長詹森表示，新海1造價新台幣6億多，
分類：大自然



In [14]:
"""
實作可以一次回傳一個 mini-batch 的 DataLoader
這個 DataLoader 吃我們上面定義的 `NewsDataset`，
回傳訓練 BERT 時會需要的 4 個 tensors：
- tokens_tensors  : (batch_size, max_seq_len_in_batch)
- segments_tensors: (batch_size, max_seq_len_in_batch)
- masks_tensors   : (batch_size, max_seq_len_in_batch)
- label_ids       : (batch_size)
"""

from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence

# 這個函式的輸入 `samples` 是一個 list，裡頭的每個 element 都是
# 剛剛定義的 `NewsDataset` 回傳的一個樣本，每個樣本都包含 3 tensors：
# - tokens_tensor
# - segments_tensor
# - label_tensor
# 它會對前兩個 tensors 作 zero padding，並產生前面說明過的 masks_tensors
def create_mini_batch(samples):
    tokens_tensors = [s[0] for s in samples]
    segments_tensors = [s[1] for s in samples]
    
    # 測試集有 labels
    if samples[0][2] is not None:
        label_ids = torch.stack([s[2] for s in samples])
    else:
        label_ids = None
    
    # zero pad 到同一序列長度
    tokens_tensors = pad_sequence(tokens_tensors, 
                                  batch_first=True)
    segments_tensors = pad_sequence(segments_tensors, 
                                    batch_first=True)
    
    # attention masks，將 tokens_tensors 裡頭不為 zero padding
    # 的位置設為 1 讓 BERT 只關注這些位置的 tokens
    masks_tensors = torch.zeros(tokens_tensors.shape, 
                                dtype=torch.long)
    masks_tensors = masks_tensors.masked_fill(
        tokens_tensors != 0, 1)
    
    return tokens_tensors, segments_tensors, masks_tensors, label_ids


# 初始化一個每次回傳 64 個訓練樣本的 DataLoader
# 利用 `collate_fn` 將 list of samples 合併成一個 mini-batch 是關鍵
BATCH_SIZE = 8
trainloader = DataLoader(trainset, batch_size=BATCH_SIZE, 
                         collate_fn=create_mini_batch)

In [15]:
data = next(iter(trainloader))

tokens_tensors, segments_tensors, \
    masks_tensors, label_ids = data


In [16]:
# 載入一個可以做中文多分類任務的模型，n_class = 10
from transformers import BertForSequenceClassification

NUM_LABELS = 10

model = BertForSequenceClassification.from_pretrained(
    PRETRAINED_MODEL_NAME, num_labels=NUM_LABELS)

clear_output()

# high-level 顯示此模型裡的 modules
print("""
name            module
----------------------""")
for name, module in model.named_children():
    if name == "bert":
        for n, _ in module.named_children():
            print(f"{name}:{n}")
    else:
        print("{:15} {}".format(name, module))


name            module
----------------------
bert:embeddings
bert:encoder
bert:pooler
dropout         Dropout(p=0.1, inplace=False)
classifier      Linear(in_features=768, out_features=10, bias=True)


In [17]:
"""
定義一個可以針對特定 DataLoader 取得模型預測結果以及分類準確度的函式
之後也可以用來生成上傳到 Kaggle 競賽的預測結果

2019/11/22 更新：在將 `tokens`、`segments_tensors` 等 tensors
丟入模型時，強力建議指定每個 tensor 對應的參數名稱，以避免 HuggingFace
更新 repo 程式碼並改變參數順序時影響到我們的結果。
"""

def get_predictions(model, dataloader, compute_acc=False):
    predictions = None
    correct = 0
    total = 0
      
    with torch.no_grad():
        # 遍巡整個資料集
        for data in dataloader:
            # 將所有 tensors 移到 GPU 上
            if next(model.parameters()).is_cuda:
                data = [t.to("cuda:0") for t in data if t is not None]
            
            
            # 別忘記前 3 個 tensors 分別為 tokens, segments 以及 masks
            # 且強烈建議在將這些 tensors 丟入 `model` 時指定對應的參數名稱
            tokens_tensors, segments_tensors, masks_tensors = data[:3]
            outputs = model(input_ids=tokens_tensors, 
                            token_type_ids=segments_tensors, 
                            attention_mask=masks_tensors)
            
            logits = outputs[0]
            _, pred = torch.max(logits.data, 1)
            
            # 用來計算訓練集的分類準確率
            if compute_acc:
                labels = data[3]
                total += labels.size(0)
                correct += (pred == labels).sum().item()
                
            # 將當前 batch 記錄下來
            if predictions is None:
                predictions = pred
            else:
                predictions = torch.cat((predictions, pred))
    
    if compute_acc:
        acc = correct / total
        return predictions, acc
    return predictions
    
# 讓模型跑在 GPU 上並取得訓練集的分類準確率
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("device:", device)
model = model.to(device)
_, acc = get_predictions(model, trainloader, compute_acc=True)
print("classification acc:", acc)

device: cuda:0
classification acc: 0.04609929078014184


In [18]:
def get_learnable_params(module):
    return [p for p in module.parameters() if p.requires_grad]
     
model_params = get_learnable_params(model)
clf_params = get_learnable_params(model.classifier)

print(f"""
整個分類模型的參數量：{sum(p.numel() for p in model_params)}
線性分類器的參數量：{sum(p.numel() for p in clf_params)}
""")


整個分類模型的參數量：102275338
線性分類器的參數量：7690



In [19]:
%%time

# 訓練模式
model.train()

# 使用 Adam Optim 更新整個分類模型的參數
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)


EPOCHS = 5  # 幸運數字
for epoch in range(EPOCHS):
    
    running_loss = 0.0
    for data in trainloader:
        
        tokens_tensors, segments_tensors, \
        masks_tensors, labels = [t.to(device) for t in data]

        # 將參數梯度歸零
        optimizer.zero_grad()
        
        # forward pass
        outputs = model(input_ids=tokens_tensors, 
                        token_type_ids=segments_tensors, 
                        attention_mask=masks_tensors, 
                        labels=labels)

        loss = outputs[0]
        # backward
        loss.backward()
        optimizer.step()


        # 紀錄當前 batch loss
        running_loss += loss.item()
        
    # 計算分類準確率
    _, acc = get_predictions(model, trainloader, compute_acc=True)

    print('[epoch %d] loss: %.3f, acc: %.3f' %
          (epoch + 1, running_loss, acc))

[epoch 1] loss: 69.444, acc: 0.518
[epoch 2] loss: 47.230, acc: 0.677
[epoch 3] loss: 33.574, acc: 0.840
[epoch 4] loss: 22.532, acc: 0.915
[epoch 5] loss: 15.812, acc: 0.947
CPU times: user 1min, sys: 39.4 s, total: 1min 40s
Wall time: 1min 40s


In [34]:
%%time
# 建立測試集。這邊我們可以用跟訓練時不同的 batch_size，看你 GPU 多大
testset = NewsDataset("test_news", tokenizer=tokenizer)
testloader = DataLoader(testset, batch_size=8, 
                        collate_fn=create_mini_batch)

# 用分類模型預測測試集
predictions = get_predictions(model, testloader)

# 用來將預測的 label id 轉回 label 文字
index_map = {v: k for k, v in testset.label_map.items()}

# 生成 Kaggle 繳交檔案
df = pd.DataFrame({"Category": predictions.tolist()})
df['Category'] = df.Category.apply(lambda x: index_map[x])
df_pred = pd.concat([testset.df.loc[:, ["id"]], 
                          df.loc[:, 'Category']], axis=1)
df_pred.to_csv('bert_1_prec_training_samples.csv', index=False)
df_pred.head()

CPU times: user 243 ms, sys: 133 ms, total: 376 ms
Wall time: 377 ms


In [35]:
valid = pd.read_csv("/content/news/valid.csv")
df_pred.merge(valid, on=['id'])
df_pred.to_csv('pred.csv', index=False, encoding='utf-8-sig')

!cat pred.csv

﻿id,Category
283,社會新聞
284,政治
285,社會新聞
286,社會新聞
287,社會新聞
288,名人
289,社會新聞
290,政治
291,天氣
292,政治
293,社會新聞
294,政治
295,名人
296,政治
297,社會新聞
298,社會新聞
299,政治
300,疫情
